In [13]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
from itertools import product

In [2]:
# env variables
REPO_PATH = '/home/ah19/runtime-monitoring'
DATASET = 'MNIST'
PREFIX = 'Adam-256-30'
FILENAME_POSTFIX = f'{DATASET}_{PREFIX}'
DATA_FALVOR = 'raw'
LOAD_NEURONS = True
POSTFIX = ''

import os
os.chdir('../..')
from utilities.utils import load_json
from utilities.pathManager import fetchPaths
from utilities.MonitorUnifiedBDD import build_bdd_multi_etas, build_bdd

In [4]:
# paths
base = Path(REPO_PATH)
paths = fetchPaths(base, DATASET)

path = paths[DATASET.lower()]
path_bdd_testingThresholds = paths['bdd_testingThresholds_' + DATA_FALVOR] / FILENAME_POSTFIX

path_lastHiddenLayer_pca = paths['lastHiddenLayer_pca']
path_lastHiddenLayer_pca_single = path_lastHiddenLayer_pca / FILENAME_POSTFIX / 'Single'
# path_lastHiddenLayer_pca_classes = path_lastHiddenLayer_pca / FILENAME_POSTFIX / 'Classes'

path_lastHiddenLayer = paths['lastHiddenLayer_raw'] / FILENAME_POSTFIX

In [5]:

# import Data
print('Loading train Data ...')
df = pd.read_csv(path_lastHiddenLayer / f'{FILENAME_POSTFIX}_train.csv')

# split train data
df_true = df[df['true'] == True].copy()
df_true = df_true.drop('true', axis=1).reset_index(drop=True)

print('Loading test Data ...')
df_test = pd.read_csv(path_lastHiddenLayer / f'{FILENAME_POSTFIX}_test.csv')


print('Loading Neurons ...')
neurons = None
if LOAD_NEURONS:
    neurons = load_json(path_lastHiddenLayer_pca_single / f'{FILENAME_POSTFIX}_neurons.json')


Loading train Data ...
Loading test Data ...
Loading Neurons ...


In [6]:
# define threshold

p = 0.9

thld = np.quantile(df_true.drop('y', axis=1), p, axis=0)
thld_name = f'qth_{p}'

# degree of freedom
eta = 0

In [7]:
res = build_bdd((df.copy(), df_test.copy(), df_true.copy(), neurons, thld_name, thld, eta, None))

qth_0.9 - eta: 0
> Done! [ qth_0.9 - eta: 0 ]


In [8]:
res_m = build_bdd_multi_etas((df.copy(), df_test.copy(), df_true.copy(), neurons, thld_name, thld, eta, None))

qth_0.9 - eta: 0
> Done! [ qth_0.9 - eta: 0 ]


In [9]:
res[0]

,thld,df,build_time,size_before_reorder_mb,reorder_time,size_after_reorder_mb
1,qth_0.9,0.0,16.146,35.547,0.68,4.012


In [10]:
res[1].loc[(res[1]['stage'] == 'test') & (res[1]['y'] == 'all'), 'outOfPatternMissClassified'].values[0]

1.0

In [11]:
res_m[0]

,thld,df,build_time,size_before_reorder_mb,reorder_time,size_after_reorder_mb
1,qth_0.9,0.0,17.426,35.547,0.69,4.012


In [12]:
res_m[1].loc[(res_m[1]['stage'] == 'test') & (res_m[1]['eta'] == eta)]

,y,_false,_false_miss_classified,count,outOfPattern,outOfPatternMissClassified,eta,stage,thld
4,2,2.0,0.0,1032.0,0.001938,1.0,0,test,qth_0.9
5,4,3.0,0.0,982.0,0.003055,1.0,0,test,qth_0.9
6,5,4.0,0.0,892.0,0.004484,1.0,0,test,qth_0.9
7,6,2.0,0.0,958.0,0.002088,1.0,0,test,qth_0.9
8,8,6.0,0.0,974.0,0.006160,1.0,0,test,qth_0.9
9,9,1.0,0.0,1009.0,0.000991,1.0,0,test,qth_0.9
10,all,18.0,0.0,10000.0,0.001800,1.0,0,test,qth_0.9
11,all_mean,0.0,0.0,10000.0,0.003119,1.0,0,test,qth_0.9


In [14]:
np.all(res_m[1].loc[(res_m[1]['stage'] == 'test') & (res_m[1]['eta'] == eta), 'outOfPatternMissClassified'] \
== res[1].loc[res[1]['stage'] == 'test', 'outOfPatternMissClassified'])

True